In [1]:
GPU_device_id = str(4)
model_id_save_as = 'learningcurve-daednn-easy-final'
architecture_id = 'final-models/learningcurve-dnn-easy-final-features'
model_class_id = 'DNN'
testing_dataset_id = '../../source-interdiction/dataset_generation/validation_dataset_200keV_log10time_100.npy'
training_dataset_id = '../../source-interdiction/dataset_generation/training_dataset_200keV_log10time_10000.npy'
difficulty_setting = 'easy'

train_sizes = [50, 100, 500, 1000, 5000, 10000, 15000, 20000,]
earlystop_patience = 10
num_epochs = 2000

In [2]:
import matplotlib.pyplot as plt
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"] = GPU_device_id


from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import FunctionTransformer, LabelBinarizer
from sklearn.model_selection import StratifiedKFold, StratifiedShuffleSplit
import tensorflow as tf
import pickle
import numpy as np
import pandas as pd
from random import choice
from tensorflow.keras.models import load_model

from numpy.random import seed
seed(5)
from tensorflow import set_random_seed
set_random_seed(5)

#### Import model, training function 

In [3]:
from annsa.model_classes import build_dnn_model, compile_model, f1
from annsa.load_dataset import load_easy, load_full, dataset_to_spectrakeys
from annsa.load_pretrained_network import load_features

Using TensorFlow backend.


## Training Data Construction

In [4]:
training_dataset = np.load(training_dataset_id)
training_spectra, training_keys = dataset_to_spectrakeys(training_dataset)

## Load testing dataset

In [5]:
testing_dataset = np.load(testing_dataset_id)
testing_spectra, testing_keys = dataset_to_spectrakeys(testing_dataset)

## Load Model

In [6]:
model_features = load_features(architecture_id)
model_features.input_dim = 1024
model_features.dropout_rate = model_features.dropout_probability
model_features.loss = tf.keras.losses.categorical_crossentropy
model_features.optimizer = tf.keras.optimizers.Adam
model_features.output_function = tf.nn.softmax
model_features.metrics = [f1]
model_features.learning_rate = model_features.learining_rate

In [14]:
dae_model = load_model('./final-models-keras/daepretrain-easy-final.hdf5')

## Train network

# Scale input data

In [43]:
training_spectra_scaled = model_features.scaler.transform(training_spectra)
testing_spectra_scaled = model_features.scaler.transform(testing_spectra)

In [44]:
earlystop_callback = tf.keras.callbacks.EarlyStopping(
    monitor='val_f1',
    patience=earlystop_patience,
    mode='max',
    min_delta=0.01,
    restore_best_weights=True)

In [45]:
mlb=LabelBinarizer()

for train_size in train_sizes:
    print('\n\nRunning through training size '+str(train_size))
    k_folds_errors = []

    sss = StratifiedShuffleSplit(n_splits=5, train_size=train_size)
    k = 0
    for train_index, _ in sss.split(training_spectra, training_keys):
        print('Running through fold '+str(k))
        training_keys_binarized = mlb.fit_transform(training_keys.reshape([training_keys.shape[0],1]))
        testing_keys_binarized = mlb.transform(testing_keys)
        
        model = compile_model(
            build_dnn_model,
            model_features)
        model_weights = model.get_weights()
        model_weights_updated = model_weights[:]
        model_weights_updated[0:2] = dae_model.get_weights()[0:2]
        model.set_weights(model_weights_updated)

        csv_logger = tf.keras.callbacks.CSVLogger('./final-models-keras/'+model_id_save_as+'_trainsize'+str(train_size)+'_fold'+str(k)+'.log')

        output = model.fit(
            x=training_spectra_scaled[train_index],
            y=training_keys_binarized[train_index],
            epochs=num_epochs,
            verbose=1,
            validation_data=(testing_spectra_scaled,
                             testing_keys_binarized),
            shuffle=True,
            callbacks=[earlystop_callback, csv_logger],
        )
        model.save('./final-models-keras/'+model_id_save_as+'_trainsize'+str(train_size)+'_fold'+str(k)+'.hdf5')
        k += 1




Running through training size 50
Running through fold 0


/home/ubuntu/anaconda3/envs/tensorflow_p36_update/lib/python3.6/site-packages/sklearn/model_selection/_split.py:1639: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


Train on 50 samples, validate on 3000 samples
Epoch 1/2000
50/50 [==============================] - 1s 14ms/sample - loss: 44.8959 - f1: 0.0000e+00 - val_loss: 43.8906 - val_f1: 0.0221
Epoch 2/2000
50/50 [==============================] - 0s 3ms/sample - loss: 43.3198 - f1: 0.0437 - val_loss: 42.6352 - val_f1: 0.0303
Epoch 3/2000
50/50 [==============================] - 0s 3ms/sample - loss: 41.8928 - f1: 0.0434 - val_loss: 41.5152 - val_f1: 0.0539
Epoch 4/2000
50/50 [==============================] - 0s 3ms/sample - loss: 40.9294 - f1: 0.0469 - val_loss: 40.4604 - val_f1: 0.0608
Epoch 5/2000
50/50 [==============================] - 0s 3ms/sample - loss: 39.8346 - f1: 0.0882 - val_loss: 39.3656 - val_f1: 0.0473
Epoch 6/2000
50/50 [==============================] - 0s 3ms/sample - loss: 38.3892 - f1: 0.0453 - val_loss: 38.1758 - val_f1: 0.0185
Epoch 7/2000
50/50 [==============================] - 0s 3ms/sample - loss: 37.0418 - f1: 0.0000e+00 - val_loss: 37.0398 - val_f1: 0.0204
Epoch 8

Epoch 5/2000
50/50 [==============================] - 0s 3ms/sample - loss: 38.6086 - f1: 0.0747 - val_loss: 38.6886 - val_f1: 0.0857
Epoch 6/2000
50/50 [==============================] - 0s 3ms/sample - loss: 37.4182 - f1: 0.1690 - val_loss: 37.8027 - val_f1: 0.0947
Epoch 7/2000
50/50 [==============================] - 0s 3ms/sample - loss: 36.7279 - f1: 0.1079 - val_loss: 37.1231 - val_f1: 0.0700
Epoch 8/2000
50/50 [==============================] - 0s 3ms/sample - loss: 35.6526 - f1: 0.1048 - val_loss: 36.2339 - val_f1: 0.0565
Epoch 9/2000
50/50 [==============================] - 0s 3ms/sample - loss: 34.7674 - f1: 0.1310 - val_loss: 35.2878 - val_f1: 0.0654
Epoch 10/2000
50/50 [==============================] - 0s 3ms/sample - loss: 33.6065 - f1: 0.1989 - val_loss: 34.4577 - val_f1: 0.0804
Epoch 11/2000
50/50 [==============================] - 0s 3ms/sample - loss: 32.8593 - f1: 0.1664 - val_loss: 33.6572 - val_f1: 0.0948
Epoch 12/2000
50/50 [==============================] - 0s 3m

50/50 [==============================] - 0s 3ms/sample - loss: 20.7961 - f1: 0.2576 - val_loss: 21.4317 - val_f1: 0.1684
Epoch 39/2000
50/50 [==============================] - 0s 3ms/sample - loss: 20.8646 - f1: 0.3166 - val_loss: 21.0680 - val_f1: 0.1990
Epoch 40/2000
50/50 [==============================] - 0s 3ms/sample - loss: 20.3952 - f1: 0.3300 - val_loss: 20.9378 - val_f1: 0.1958
Epoch 41/2000
50/50 [==============================] - 0s 3ms/sample - loss: 20.3585 - f1: 0.3022 - val_loss: 20.8749 - val_f1: 0.1942
Epoch 42/2000
50/50 [==============================] - 0s 3ms/sample - loss: 19.8789 - f1: 0.3256 - val_loss: 20.7101 - val_f1: 0.1912
Epoch 43/2000
50/50 [==============================] - 0s 3ms/sample - loss: 19.7274 - f1: 0.3542 - val_loss: 20.4326 - val_f1: 0.1983
Epoch 44/2000
50/50 [==============================] - 0s 3ms/sample - loss: 19.7387 - f1: 0.3124 - val_loss: 20.1606 - val_f1: 0.2025
Epoch 45/2000
50/50 [==============================] - 0s 3ms/sample 

...

15000/15000 [==============================] - 2s 115us/sample - loss: 0.8509 - f1: 0.7855 - val_loss: 0.7485 - val_f1: 0.8000
Epoch 9/2000
15000/15000 [==============================] - 2s 115us/sample - loss: 0.8555 - f1: 0.7812 - val_loss: 0.6811 - val_f1: 0.8256
Epoch 10/2000
15000/15000 [==============================] - 2s 116us/sample - loss: 0.7936 - f1: 0.8052 - val_loss: 0.6692 - val_f1: 0.8334
Epoch 11/2000
15000/15000 [==============================] - 2s 116us/sample - loss: 0.7602 - f1: 0.8177 - val_loss: 0.5813 - val_f1: 0.8741
Epoch 12/2000
15000/15000 [==============================] - 2s 116us/sample - loss: 0.7203 - f1: 0.8315 - val_loss: 0.5386 - val_f1: 0.8943
Epoch 13/2000
15000/15000 [==============================] - 2s 117us/sample - loss: 0.7221 - f1: 0.8283 - val_loss: 0.6060 - val_f1: 0.8517
Epoch 14/2000
15000/15000 [==============================] - 2s 116us/sample - loss: 0.6904 - f1: 0.8380 - val_loss: 0.5283 - val_f1: 0.8870
Epoch 15/2000
15000/15000 [=

15000/15000 [==============================] - 2s 119us/sample - loss: 0.6283 - f1: 0.8631 - val_loss: 0.6151 - val_f1: 0.8567
Epoch 14/2000
15000/15000 [==============================] - 2s 118us/sample - loss: 0.6098 - f1: 0.8710 - val_loss: 0.4836 - val_f1: 0.9089
Epoch 15/2000
15000/15000 [==============================] - 2s 118us/sample - loss: 0.6035 - f1: 0.8741 - val_loss: 0.5268 - val_f1: 0.8913
Epoch 16/2000
15000/15000 [==============================] - 2s 118us/sample - loss: 0.6395 - f1: 0.8637 - val_loss: 0.5098 - val_f1: 0.9004
Epoch 17/2000
15000/15000 [==============================] - 2s 118us/sample - loss: 0.5965 - f1: 0.8738 - val_loss: 0.5013 - val_f1: 0.9080
Epoch 18/2000
15000/15000 [==============================] - 2s 118us/sample - loss: 0.5585 - f1: 0.8901 - val_loss: 0.4867 - val_f1: 0.9030
Epoch 19/2000
15000/15000 [==============================] - 2s 119us/sample - loss: 0.5571 - f1: 0.8881 - val_loss: 0.4518 - val_f1: 0.9252
Epoch 20/2000
15000/15000 [

15000/15000 [==============================] - 2s 124us/sample - loss: 0.8310 - f1: 0.8190 - val_loss: 0.6985 - val_f1: 0.8477
Epoch 7/2000
15000/15000 [==============================] - 2s 139us/sample - loss: 0.7712 - f1: 0.8310 - val_loss: 0.6065 - val_f1: 0.8951
Epoch 8/2000
15000/15000 [==============================] - 2s 124us/sample - loss: 0.7182 - f1: 0.8473 - val_loss: 0.6258 - val_f1: 0.8640
Epoch 9/2000
15000/15000 [==============================] - 2s 132us/sample - loss: 0.7127 - f1: 0.8473 - val_loss: 0.6444 - val_f1: 0.8561
Epoch 10/2000
15000/15000 [==============================] - 2s 129us/sample - loss: 0.6744 - f1: 0.8569 - val_loss: 0.5575 - val_f1: 0.8992
Epoch 11/2000
15000/15000 [==============================] - 2s 127us/sample - loss: 0.6645 - f1: 0.8591 - val_loss: 0.5537 - val_f1: 0.8869
Epoch 12/2000
15000/15000 [==============================] - 2s 126us/sample - loss: 0.6306 - f1: 0.8693 - val_loss: 0.5313 - val_f1: 0.9036
Epoch 13/2000
15000/15000 [===

20000/20000 [==============================] - 3s 126us/sample - loss: 0.6848 - f1: 0.8430 - val_loss: 0.5240 - val_f1: 0.8909
Epoch 11/2000
20000/20000 [==============================] - 3s 126us/sample - loss: 0.6599 - f1: 0.8476 - val_loss: 0.5247 - val_f1: 0.8891
Epoch 12/2000
20000/20000 [==============================] - 3s 127us/sample - loss: 0.6600 - f1: 0.8453 - val_loss: 0.5063 - val_f1: 0.9026
Epoch 13/2000
20000/20000 [==============================] - 3s 126us/sample - loss: 0.6561 - f1: 0.8499 - val_loss: 0.4533 - val_f1: 0.9215
Epoch 14/2000
20000/20000 [==============================] - 3s 127us/sample - loss: 0.6443 - f1: 0.8509 - val_loss: 0.4742 - val_f1: 0.9026
Epoch 15/2000
20000/20000 [==============================] - 3s 126us/sample - loss: 0.6057 - f1: 0.8628 - val_loss: 0.5784 - val_f1: 0.8511
Epoch 16/2000
20000/20000 [==============================] - 2s 123us/sample - loss: 0.6024 - f1: 0.8658 - val_loss: 0.4823 - val_f1: 0.9101
Epoch 17/2000
20000/20000 [

20000/20000 [==============================] - 2s 124us/sample - loss: 0.5279 - f1: 0.8945 - val_loss: 0.4524 - val_f1: 0.9252
Epoch 16/2000
20000/20000 [==============================] - 2s 125us/sample - loss: 0.5514 - f1: 0.8860 - val_loss: 0.5286 - val_f1: 0.8853
Epoch 17/2000
20000/20000 [==============================] - 3s 125us/sample - loss: 0.5295 - f1: 0.8928 - val_loss: 0.4008 - val_f1: 0.9462
Epoch 18/2000
20000/20000 [==============================] - 2s 123us/sample - loss: 0.5185 - f1: 0.8951 - val_loss: 0.4272 - val_f1: 0.9255
Epoch 19/2000
20000/20000 [==============================] - 3s 132us/sample - loss: 0.5167 - f1: 0.8952 - val_loss: 0.3988 - val_f1: 0.9389
Epoch 20/2000
20000/20000 [==============================] - 3s 129us/sample - loss: 0.5311 - f1: 0.8911 - val_loss: 0.4334 - val_f1: 0.9190
Epoch 21/2000
20000/20000 [==============================] - 3s 125us/sample - loss: 0.5387 - f1: 0.8896 - val_loss: 0.3780 - val_f1: 0.9491
Epoch 22/2000
20000/20000 [